# 一、OpenCUA整体框架

Computer-use Agent(CUA)由视觉-语言模型（VLMs）驱动，用于自主完成计算机任务。

OpenCUA框架包含：
1. 标注工具：一个无缝捕获人类computer-use演示的标注工具。
2. <b style="color:red">AGENTNET</b>数据集：第一个跨越3个操作系统(Windows、macOS、Linux)和200+应用程序及网站的大规模computer-use任务数据集。
3. 一个可扩展的pipeline：将演示转换为带有CoT推理的状态-动作对，随数据规模的增长而提升性能。

注:
- OSWorld：一个在线评估基准，用来测试AI智能体在<b style="color:orange">真实计算机环境</b>中完成任务的能力。



![OpenCUA框架图](./images/OpenCUA%20Framework.png)

## 1. AGENTNET Tool（数据采集工具）

记录人类操作电脑的过程：屏幕录像、鼠标键盘操作记录、可访问性树（AxTree）-页面元素结构、HTML网页内容、时间戳

关键：可在Windows/Linux/MacOS个人电脑上后台运行，不打扰用户正常工作。

## 2. AGENTNET Method（数据处理方法）

把原始录像转换成模型能学习的格式，包含三大核心技术：

1. Action Reduction（动作降维）：原始记录有成千上万个细粒度动作，使用规则合并成高级动作。
2. State-action Matching（动作状态匹配）：动作离散化。从视频中提取关键帧，确保截图是“动作执行前”的状态，把每个动作和对应的屏幕截图配对。
3. <b style="color:orange">Reflector</b>（反思器）：LLM分析每一步是否正确，输出三种CoT：
    - L3：Observation + Thought + Action
    - L2：Thought + Action
    - L1：Action

注：由于扩展智能体动作需要用反思性的长CoT推理来增强这些轨迹，因此引入Reflector。即不仅让模型看“人类做了什么动作”，还得加入推理过程，教他“为什么这么做“、”做错了怎么办“。

## 3. AGENTNET Dataset & Bench（数据集和基准）

- AGENTNET Dataset：包含22625条人类操作轨迹，覆盖140+应用、190+网站，平均每条轨迹18.6步
- AGENTNET Bench：100个代表性任务，用于快速离线评估，每一步提供多个正确答案选项

## 4. OpenCUA Models（模型训练）

训练流程：
- 输入：任务指令 + 历史观察 + 当前截图
- 输出：思考过程 + 下一步动作

训练策略：混合L1/L2/L3三种CoT格式；混合GUI定位数据、规划数据、通用SFT数据

评估：在OSWorld中执行任务，测试成功率

# **二、AGENTNET数据收集**

OpenCUA的目标：在多样化的计算机环境和用户场景中扩展桌面computer-use数据。

为提高数据收集的可扩展性，优先收集遵循自然用户行为的演示，对用户与计算机的交互方式施加最少的额外约束。

→ Solution：开发 <b style="color:orange">AGENTNET Tool</b>，收集数据得到AGENTNET Dataset。

## 1. 任务定义

人类动作与智能体动作的对应关系：
| **人类动作** | **动作描述** | **智能体动作函数** |
|-------|-------|-------|
| Click | 在特定位置点击 | `click(x, y, button)`|
| Middle Click | 在特定位置中键点击 | `middleClick(x, y)` |
| Double Click | 在特定位置双击 | `doubleClick(x, y, button)` |
| Triple Click | 在特定位置三击 | `tripleClick(x, y, button)` |
| Mouse Move | 移动鼠标到特定位置 | `moveTo(x, y)` |
| Drag | 拖拽鼠标从一个位置到另一个位置 | `dragTo(x, y)` |
| Scroll | 处置或水平滚动 | `scroll(dx, dy)`/`hscroll(dx, dy)` |
| Type | 输入一串长文 | `write(text)` |
| Press | 按下特定按键 | `press(key)` |
| Hotkey | 执行组合键 | `hotkey(key1, key2)` |
| Wait | 等待几秒 | `wait()` |
| Terminate | 结束任务（成功或失败） | `terminate('success', 'failure')` |

<br>

**任务建模方式**：智能体的决策过程被建模为**状态-动作转换轨迹**`(I, <s₀, a₀>, <s₁, a₁>, ..., <sₜ, aₜ>)`  
- `I`：任务语言执行
- `s₀`：初始状态（屏幕截图）
- `a₀`：第一个动作
- 智能体按顺序预测动作`aᵢ`直至目标状态`aₜ`，最后执行终止动作`aₜ`：`terminate('success')`或`terminate('failure')`  
    预测：给定任务指令和之前的所有状态-动作历史，预测下一个动作`P(aᵢ | I, s₀, a₀, ..., sᵢ)`

**计算机任务状态的表示**：<b style="color:green">使用计算机的<b style="color:orange">屏幕截图</b>作为智能体的<b style="color:orange">observation</b></b>。  
> Q：为什么是截图，而不是其他信息？  
> A：传统方法用HTML/DOM结构（只适用于网页）、AxTree（有些应用不支持），但**屏幕截图最通用**，适合所有应用。

**动作空间设计**：基于PyAutoGUI的动作子集，确保跨OS兼容，并添加智能体专用动作`terminate`。

## 2. <b style="color:orange">AGENTNET Tool</b>

用于收集computer-use演示。  
1. 在标注者的个人计算机上后台运行、记录，捕获：屏幕视频、鼠标键盘信号、可访问性树（AxTree）。
    - 鼠标和键盘输入跟踪：DuckTrack、OpenAdapt
    - 屏幕录制：OBS Studio
    - AxTree解析：遵循OSWorld框架。
2. 然后将这些数据处理成状态-动作轨迹，允许标注者：审查、编辑和提交演示，描述总体目标的任务指令。

### · 标注流程（Annotation pipeline）

数据收集的关键目标：**多样性**、**复杂性**  
任务要求：任务必须超过15步；少于5步的任务会被拒绝。  
所有标注结果经过人工check，根据目标清晰度、多样性和复杂性标记为：rejected、ok、good、excellent。

### · 构建紧凑的状态-动作轨迹

原始演示由高频屏幕录像和细粒度交互信号（鼠标移动、点击、滚动、按键）组成，单个任务包含数千个低级动作，无法训练。  
**Solution：**
1. <b style="color:red">动作降维（Action reduction）</b>：基于规则，将原子信号压缩成更高级的操作，同时保留并基本的动作信息。→ 简化的动作序列（与pyautogui动作空间对齐）  
    - 鼠标移动事件：被视为点击或拖拽的前置条件，只保留起始和结束位置。
    - 滚动：被合并成单向动作，累计滚轮计数。
    - 连续按键：被合并成文本输入字符串，修饰键组合（比如CTRL+C）被抽象为热键动作。
    - 组合常见的多步手势，比如拖拽或双击。
2. <b style="color:red">状态-动作匹配（State-action matching）</b>：将每个动作与代表性状态配对。从屏幕录像中提取**关键帧**，这些关键帧捕获动作发生前的系统状态。在最后一个动作之后，附加一个终止帧以及相应的终止动作。
    > Q：将关键帧与鼠标点击的动作时间戳对齐会有<b style="color:blue">Data Leakage风险</b>。比如鼠标可能已经定位在按钮上，模型只需要看到鼠标位置就输出一个click动作；但模型没学会怎么找到按钮、为什么要点击这个按钮，怎么从屏幕其他地方导航到这里。  
    > Solution：对于鼠标点击，回溯到鼠标开始移动前；向后搜索以找到最后一个视觉上不同的帧。这样模型才能识别按钮在哪、理解需要点击它、计算出正确的坐标。  

    注：
    - <u>Data Leakage</u>：训练数据中包含了<u>预测时不应该知道的未来信息</u>。  
        后果：模型在训练集上表现很好，但在真实环境中完全失效。因为它学到的是“作弊技巧”，而不是真正的技能。

![AGENTNET Tool标注与校验](./images/AGENTNET%20Tool.png)



# <b style="color:red">三、训练CUA模型</b>

AGENTNET包含`I`（任务指令）和`<sᵢ, aᵢ>`（状态-动作）轨迹，但<u>直接在这些轨迹上微调 视觉-语言-动作（VLA）模型的性能很差</u>。  
Solution：一种新颖的推理增强方法、上下文编码技术、数据混合技术

## 1. <b style="color:orange">合成反思性长链CoT推理（三层推理结构L3→L2→L1）</b>

自然语言推理对于可泛化的computer-use基础模型很重要，帮助CUA内化认知能力。→ **多阶段CoT框架**：为每个`<sᵢ, aᵢ>`合成结构化推理。  
- <b style="color:orange">L3 - 上下文观察（observation）</b>：捕获突出的视觉和文本元素。
- <b style="color:orange">L2 - 反思性推理（Thought）</b>：分析状态转换、回忆之前的步骤、纠正错误、规划后续动作。  
    **L2推理的反思增强**：<u>人类演示中的不正确或冗余标注并非全是坏事</u>，只要我们能识别并使用它们来教模型识别和纠正错误。 → <b style="color:orange">Reflector：识别错误并为每一步生成反思推理。</b>

    CoT合成pipeline的三大组件：<b style="color:orange">reflector（反思器）、generator（生成器）和summarizer（总结器）</b>。  
        合成流程：Generator和Reflector通过爹爹带生成和验证观察与真实行动之间的推理组件。

    1. Reflector：<u>检查每一步的正确性和必要性</u>。
        - 检查方式：
            - 比较动作前后的截图
            - 检查动作代码本身的正确性
            - 检查生成的CoT
            - 特别关注：**“Action”是否与截图和代码对齐**
        - 输出：
            - 如果步骤不正确或冗余：详细说明原因；这一步将在训练时被忽略。
            - 如果步骤正确：解释动作前后状态带来的差异。
    2. Generator：<u>基于完整智能体上下文生成结构化的CoT</u>
        - 输入：之前的反思 + 动作历史 + 任务目标 + 截图 + 动作代码
        - 输出：结构化的CoT
        - 视觉辅助：为帮助模型更准确定位坐标相关的动作，加入视觉提示。
            - 在鼠标动作坐标上放一个**红色标记**；
            - 提供一个**放大的图像块**。
    3. Summarizer：
        - 将用户写的模糊任务目标改成更精确、更对齐的任务描述
        - 为每条轨迹打分（对齐度、效率、难度）

    ![反思性长链CoT合成流程](./images/reflective%20long%20CoT%20synthesis.png)
- <b style="color:orange">L1 - 可执行动作（Action）</b>：基于先前感知和思考，预测简洁可执行动作。

### 示例

在预测实际行动前，模型首先基于历史记录与当前行动，通过CoT中的反思、记忆、规划与推理进行推演，识别先前步骤中的错误，并在后续阶段进行修正。

![反思性长链CoT推理示例](./images/reflective%20long%20CoT%20example.png)

<br>
<br>

## 2. <b style="color:orange">智能体在执行任务时，怎么记住之前做过什么</b>

假设任务：在word中创建一个5行2列的表格。

```text
    Step1：打开Word
    Step2：点击“插入”菜单
    Step3：点击“表格”
    Step4：输入行数“5” ← 当前步骤
```

Question：到了Step4，模型怎么知道：我为什么要输入数字、我之前点了什么、任务目标是什么  
Answer：<b style="color:orange">需要把<u>历史信息</u>编码到模型的输入中！</b>

### <b style="color:orange">1) 文本历史：用文字记录之前做了什么</b>

表示历史的方式：  
1. 完整的L3 CoT：太长！放不下很长的历史
```text
    Step1:
        Observation：看到桌面，有Word图标...
        Thought：我需要打开Word，因为任务要创建表格...
        Action：doubleClick(Word图标)
    Step2：
        Observation：Word打开了，看到菜单栏...
        Thought：我需要点击插入表格，所以点击“插入”...
        Action：click(插入菜单)
    Step3：...
```
2. <b style="color:orange">只保留L1 CoT</b>：简洁！OpenCUA的选择，动作序列已经说明了“做了什么”
```text
    Step1: doubleClick(Word图标)
    Step2: click(插入菜单)
    Step3: click(表格)
    Step4: 现在输入行数...
```

BUT，<b style="color:orange">只有L1会不会丢信息？</b> → Solution：在当前步骤的CoT中加入<b style="color:orange">Memory组件</b>
- <b style="color:orange">历史用简短的L1表示</b>（节省空间）
- <b style="color:orange">当前步骤用详细的L2表示</b>（包含Memory，补偿历史信息）  
```text
    Step4:
        Memory:
            Step1: 打开Word
            Step2: 点击插入菜单
            Step3: 点击表格
            任务目标: 创建5行2列的表格
        Thought: 现在在表格设置对话框，需要输入行数“5”
        Action: write("5")
```

### <b style="color:orange">2) 视觉历史：用截图记录之前的画面</b>

Question：为什么需要视觉历史？  
Answer：文字描述又是不够准确，<b style="color:orange">截图是无损的！</b>

Question：放几张截图？  
Answer：<b style="color:orange">3张截图！</b>不是越多越好，多放截图信息更完整吗，但输入变长，训练变慢。

## 3. <b style="color:orange">训练时vs测试时的推理格式</b>

- <b style="color:orange">训练时</b>：<b style="color:orange">混合使用L1/L2/L3三种格式</b>，让模型学会不同层次的推理
- <b style="color:orange">测试时</b>：<b style="color:orange">统一使用L2 CoT格式</b>
    - Question：为什么L2最好？
    - Answer：
        - L1：太简单，没有推理
        - L2：
            - 刚刚好，有规划、反思，直接指向动作
            - 可扩展性强！→ <b style="color:green">Pass@n 评估</b>（让模型生成n次，只要有一次成功就算成功）
        - L3：太啰嗦，observation包含无关干扰信息

## 4. <b style="color:orange">训练数据应该怎么配比？</b>

<b style="color:red">Core Thought: 不能只喂computer-use数据，要混合通用知识，模型才能真正聪明！</b>

如果只用computer-use数据训练，模型会：
1. 变成“操作机器人”，不会推理
2. 不理解为什么要这么做
3. 遇到新场景就懵了

### <b style="color:orange">OpenCUA Solution: 三种数据混合</b>

1. <b style="color:orange">认识界面元素（Grounding）</b>
    - 教材：
        ```text
        屏幕上这个蓝色按钮是“提交”按钮
        左上角的图标是“文件”菜单
        输入框在屏幕中央，坐标是(500, 300)
        ```
    - OpenCUA Data：
        - ShowUI：GUI元素定位数据
        - UGround：通用定位数据
        - 189K边界框样本：从AxTree解析出来的元素位置
    - 目的：教会模型<b style="color:orange">看懂界面</b>
        - 能识别按钮、菜单、输入框
        - 知道每个元素在哪
2. <b style="color:orange">执行复杂任务（Planning）</b>
    - 教材：
        ```text
        任务：在Word中创建5行2列的表格
        步骤：
            1. 打开Word
            2. 点击“插入”
            3. 点击”表格“
            4. 输入行数“5”，列数“2”
            5. 点击“确定”
        ```
    - OpenCUA Data：
        - 18K Windows/MacOS轨迹
        - 10K Ubuntu轨迹
        - 任务指令增强样本（Reflector生成）
    - 目的：教会模型<b style="color:orange">规划多步任务</b>
        - 把大任务拆解成小步骤
        - 按正确顺序执行
        - 能检测和纠正错误
3. <b style="color:orange">通用知识和梳理（通用SFT）</b>
    - 教材：
        ```text
        如何写一封正式邮件？
        1+1等于几？
        总结这篇长文档的要点
        识别图片中的文字
        ```
    - OpenCUA Data：（From Kimi Team）
        - 通用文本数据：指令遵循、数学推理、长文本推理
        - 通用视觉数据：OCR、Vision QA
    - 目的：让模型不只是”操作机器人“，而是<b style="color:orange">”聪明的助手“</b>
        - 理解用户意图
        - 做推理和判断
        - 处理各种场景

Question: 为什么要混合三种数据？  
Answer: 实验证明，加入通用数据后，性能提升。

Question: 为什么通用数据有用？  
Answer: 通用数据包含了模型没有见过的场景，加入后，模型可以泛化到新场景。

#### <b style="color:red">⭐核心：CoT混合、任务混合、训练策略</b>

训练时，混合使用L1/L2/L3三种CoT，让模型学会不同层次的推理。
- L1：学会”动作和效果的直接关系“
- L2：学会”规划和推理“
- L3：学会”观察和理解“

<b style="color:orange">三种训练策略：</b>
1. <b style="color:orange">策略1：只训练Stage2 → 快速原型</b>
    - 适用场景：预算有限，快速训练
    - 流程：用现成的通用VLM，直接在CUA数据上微调，得到专门的计算机使用智能体
    - 数据配比：
        - 70% CUA Data（Planning:Grounding = 4:1）
        - 30% 通用SFT数据
2. <b style="color:orange">策略2：Stage1+Stage2（效果最好！） → 专用CUA</b>
    - 适用场景：预算充足，追求最佳效果
    - 流程：
        - Stage1 - 先学”看懂界面“：大量定位数据 + 通用VL任务 + 通用文本数据 → 训练40B tokens
        - Stage2 - 再学”规划任务“：45%规划数据 + 20%定位数据 + 35%通用数据 → 训练60B tokens
    - OpenCUA-72B的特殊处理：用额外的8K高质量轨迹（CoT质量极高）
        - Stage1 - 学各种行为：250B tokens
        - Stage2 - 把行为“压缩”成高效CoT：16B tokens
3. <b style="color:orange">策略3：联合训练 → 通用助手</b>
    - 适用场景：目标是通用VLM+CUA能力
    - 流程：不分阶段，所有数据一起训练 → 得到既能做通用任务、又能当计算机助手的模型
    - 数据配比：
        - 20% Planning data
        - 20% Grounding data
        - 60% 通用数据
    - 特殊处理：多图像轨迹数据训练3个epoch（这类数据更复杂，需要多看几遍才能学会）
    - 训练量：200B tokens，128张A100训练8天









